# Lesson3 系列データで分類・予測させてみよう（RNN, LSTM）

## Homework

RNNを用いてさらに高精度なECG5000の分類器を作ってみましょう。

ネットワークの形などは特に制限を設けませんし、今回のLessonで扱った内容以外の工夫も組み込んでもらって構いません。

上位者はリーダーボードに掲載させていただきます。（評価はaccuracyによって行います。）

### 目標値
Accuracy 95%

### ルール
- 訓練データは`x_train`, `y_train`, テストデータは`x_test`で与えられます.
- 予測ラベルは **one_hot表現ではなく0~4のクラスラベル** で表してください.
- 下のセルで指定されているx_train, y_train以外の学習データは使わないでください.

### 評価について

- テストデータ(x_test)に対する予測ラベルをcsvファイルで提出してください.
- ファイル名はsubmission.csvとしてください.
- 予測ラベルのy_testに対する精度 (F値)で評価します.
- 毎日24時にテストデータの一部に対する精度でLeader Boardを更新します.
- 最終的な評価はテストデータ全体に対する精度でおこないます.

### サンプルコード
**次のセルで指定されているx_train, y_trainのみを使って学習させてください.**

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

def load_dataset():
    # 学習データ
    x_train = np.load('/root/userspace/public/lesson3/data/x_train.npy')
    y_train = np.load('/root/userspace/public/lesson3/data/y_train.npy')
    y_train = to_categorical(y_train[:, np.newaxis], num_classes = 5)
    
    # テストデータ
    x_test = np.load('/root/userspace/public/lesson3/data/x_test.npy')

    return (x_train, x_test, y_train)

x_train, x_test, y_train = load_dataset()

In [ ]:
import tensorflow.python.keras
from keras.models import Sequential
from keras.layers import Dense, Activation, SimpleRNN
import pandas as pd

hid_dim = 10

model = Sequential()

model.add(SimpleRNN(hid_dim, input_shape=x_train.shape[1:])) # input_shape=(系列長T, x_tの次元), output_shape=(系列長T, units(=hid_dim))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(x_train, y_train, epochs=50, batch_size=100, verbose=2, validation_split=0.2)

y_pred = np.argmax(model.predict(x_test), 1)

submission = pd.Series(y_pred, name='label')
submission.to_csv('/root/userspace/submission.csv', header=True, index_label='id')